In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from tqdm.auto import tqdm
import random
import datetime
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DATA_PATH = '/content/drive/Shareddrives/dacon/404/'
SAVE_PATH ='/content/drive/Shareddrives/dacon/404/'
tre = pd.read_csv(DATA_PATH+"train_err2.csv")
tse = pd.read_csv(DATA_PATH+"test_err2.csv")

tre.time =  pd.to_datetime(tre.time)
tse.time =  pd.to_datetime(tse.time)

In [4]:
tp=pd.read_csv(DATA_PATH+"train_prob2.csv")


In [5]:
tp.time = pd.to_datetime(tp.time)

In [6]:
p_user = tp.user_id.unique()
np_user = list(set(tre.user_id.unique())  - set(p_user))

In [7]:
def check_unique(col,df1,df2):
    def change_len(x):
        if len(x) ==10:
            return x[:5]
        else:
            return x
    print("about",col)
    if col !='fwvers':
        train_c = set(df1[col].unique())
        test_c  = set(df2[col].unique())
        print("train only in %s "%col,train_c-test_c)
        print("test only in %s "%col,test_c -train_c)
        total = (train_c | test_c)
    else:
        train_c = set(df1[col].apply(change_len).unique())
        test_c  = set(df2[col].apply(change_len).unique())
        print("train only in %s "%col,train_c-test_c)
        print("test only in %s "%col,test_c -train_c)
        total = (train_c | test_c)        
        
    print()
        
    return total

model_total=check_unique('model_nm',tre,tse)
errtype_total=check_unique('errtype',tre,tse)
fwver_total = check_unique('fwver',tre,tse)

about model_nm
train only in model_nm  set()
test only in model_nm  set()

about errtype
train only in errtype  set()
test only in errtype  set()

about fwver
train only in fwver  {'04.33.1095', '04.16.3345', '04.22.1442', '04.16.2641', '05.15.2122', '05.15.2090'}
test only in fwver  {'04.22.1170', '04.22.1478', '10.22.1780', '04.22.1772', '04.22.1448', '10.22.1770', '04.73.2577', '04.22.1608', '04.73.2569'}



In [8]:
pd.options.display.max_columns=50
pd.options.display.max_rows=200

In [63]:
def get_distribution_errtype():
    total=[]
    for num in range(1,43):
        t1=tre.loc[tre.errtype==num]
        
        t11=t1.user_id.value_counts().describe().to_frame().T.rename(index={'user_id':num})
        pnumber=len(t1.loc[t1.user_id.isin(p_user)].user_id.unique())
        npnumber=len(t1.loc[t1.user_id.isin(np_user)].user_id.unique())
        
        
        t11['p_counts'] =pnumber
        t11['npnumber'] =npnumber
        if pnumber+npnumber ==0:
            t11['p_ratio']  = round((pnumber/1)*100,2)
        else:
            t11['p_ratio']  = round((pnumber/(pnumber+npnumber))*100,2)
            
        total.append(t11)

    
    return pd.concat(total).reset_index().rename(columns={'index':'errtype'})

errtpye_distribution = get_distribution_errtype()

errtpye_distribution

,errtype,count,mean,std,min,25%,50%,75%,max,p_counts,npnumber,p_ratio
0,1,950.0,22.678947,18.943070,1.0,2.0,30.0,30.0,171.0,242,708,25.47
1,2,390.0,66.938462,352.626788,1.0,2.0,2.0,6.0,4056.0,168,222,43.08
2,3,1679.0,18.319238,210.186901,1.0,1.0,2.0,5.0,7284.0,465,1214,27.70
3,4,6881.0,188.453713,2778.822793,1.0,1.0,17.0,152.0,195228.0,2298,4583,33.40
4,5,11010.0,86.550409,261.247025,1.0,3.0,9.0,49.0,6637.0,4028,6982,36.58
5,6,6467.0,7.806092,47.292207,1.0,1.0,2.0,4.0,1796.0,2662,3805,41.16
6,7,7648.0,6.798248,46.755629,1.0,1.0,2.0,4.0,2687.0,3093,4555,40.44
7,8,64.0,2.593750,2.549315,1.0,1.0,2.0,3.0,19.0,13,51,20.31
8,9,63.0,1.730159,2.444258,1.0,1.0,1.0,1.0,17.0,38,25,60.32
9,10,2360.0,56.526695,1064.242805,1.0,1.0,2.0,6.0,49106.0,1134,1226,48.05


In [64]:
def make(num):
    err1=tre.loc[tre.errtype==num]
    users=err1.user_id.value_counts().to_frame().index
    t2 =err1.loc[err1.user_id.isin(users)]
    t2['err_'+str(num)] = t2.time.dt.hour

    p = t2.loc[t2.user_id.isin(p_user)]
    np = t2.loc[t2.user_id.isin(np_user)]

    pdf =p['err_'+str(num)].value_counts().sort_index().to_frame().T.rename(index={'err_'+str(num):'err_p_'+str(num)})
    npdf =np['err_'+str(num)].value_counts().sort_index().to_frame().T.rename(index={'err_'+str(num):'err_np_'+str(num)})

    f = pd.concat([pdf,npdf])
    f2= f.copy()
    f=f.fillna(0)
#     X_MinMax_scaled = minmax_scale(f, axis=1, copy=True)
#     f.iloc[:,:]=X_MinMax_scaled*100
    f=round(f)
    return f,f2

In [65]:
t = []
for v in range(1,43):
    if v==29:
        pass
    else:
        
        df,_ = make(v)
        t.append(df)


tt = pd.concat(t).fillna(0)

tv=tt.T.corr()

tv1=tv.loc[tv.index.str.contains('np')].iloc[:,::2]

d =[]
for v in range(41):
    if v==28:
        d.append(0)
        vv=tv1.iloc[v,v]
    else:  
        vv=tv1.iloc[v,v]
    d.append(vv)
codf = pd.DataFrame(d).reset_index().rename(columns={'index':'errtype',0:'n_np_cor_hour_counts'})
codf['errtype'] =codf['errtype']+1

In [12]:
pd.merge(codf,errtpye_distribution)

,errtype,n_np_cor_hour_counts,count,mean,std,min,25%,50%,75%,max,p_counts,npnumber,p_ratio
0,1,0.996303,950.0,22.678947,18.943070,1.0,2.0,30.0,30.0,171.0,242,708,25.47
1,2,0.396358,390.0,66.938462,352.626788,1.0,2.0,2.0,6.0,4056.0,168,222,43.08
2,3,0.301628,1679.0,18.319238,210.186901,1.0,1.0,2.0,5.0,7284.0,465,1214,27.70
3,4,0.762725,6881.0,188.453713,2778.822793,1.0,1.0,17.0,152.0,195228.0,2298,4583,33.40
4,5,0.975115,11010.0,86.550409,261.247025,1.0,3.0,9.0,49.0,6637.0,4028,6982,36.58
5,6,-0.237164,6467.0,7.806092,47.292207,1.0,1.0,2.0,4.0,1796.0,2662,3805,41.16
6,7,0.440525,7648.0,6.798248,46.755629,1.0,1.0,2.0,4.0,2687.0,3093,4555,40.44
7,8,-0.040990,64.0,2.593750,2.549315,1.0,1.0,2.0,3.0,19.0,13,51,20.31
8,9,0.566893,63.0,1.730159,2.444258,1.0,1.0,1.0,1.0,17.0,38,25,60.32
9,10,0.062688,2360.0,56.526695,1064.242805,1.0,1.0,2.0,6.0,49106.0,1134,1226,48.05


In [13]:
def preprocessing_fw_update(target_df,mode='train'):
    if mode =='train':
        first_num = 10000
        count_num =15000
        
    else:
        first_num = 30000
        count_num =14999   
        
        
    dp = target_df[['user_id','model_nm','fwver']]

    unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

    fwver_total_dic ={}
    for v in range(len(fwver_total)):
        fwver_total_dic[sorted(list(fwver_total))[v]] = v+1

    def fwver_tran(x):
        return fwver_total_dic[x]

    fwver = np.zeros((count_num,21))
    for idx in tqdm(unique_data.user_id.unique()):
        df = unique_data.loc[unique_data.user_id==idx].fwver.values
        for l in range(len(df)):
            fwver[idx-first_num,l] =fwver_total_dic[df[l]]

    fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

    fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
    fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
    fwver_total_dic_rev[0] =0
    fwver_total_dic_rev2[0] = '04.22.1750'  #max 값
    

    def fwver_tras_reverse(x):
        return fwver_total_dic_rev[x]
    
    def fwver_tras_reverse2(x):
        return fwver_total_dic_rev2[x]
    
    fw_df[0] =fw_df[0].apply(fwver_tras_reverse2)
    fw_df[1] =fw_df[1].apply(fwver_tras_reverse)
    fw_df[2] =fw_df[2].apply(fwver_tras_reverse)
    fw_df[3] =fw_df[3].apply(fwver_tras_reverse)
    fw_df[4] =fw_df[4].apply(fwver_tras_reverse)

    fw_df['user_id'] +=first_num

    def model(x):
        if x in p_user:
            return 1
        else:
            return 0




    if mode =='train':
        fw_df['pb'] =fw_df.user_id.apply(model)

        fw_df2=fw_df.loc[fw_df.pb ==1]
        pfw = fw_df2[[0,1,2,3,4]].value_counts().to_frame().rename(columns={0:'problem'}).reset_index().sort_values(0)\
        .rename(columns={0:'fw1',1:'fw2',2:'fw3',3:'fw4',4:'fw5'}).reset_index(drop=True)

        fw_df2=fw_df.loc[fw_df.pb ==0]
        npfw = fw_df2[[0,1,2,3,4]].value_counts().to_frame().rename(columns={0:'non-problem'}).reset_index().sort_values(0)\
        .rename(columns={0:'fw1',1:'fw2',2:'fw3',3:'fw4',4:'fw5'}).reset_index(drop=True)

        fw_t= fw_df[[0,1,2,3,4]].value_counts().to_frame().rename(columns={0:'counts'}).reset_index().sort_values(0)\
        .rename(columns={0:'fw1',1:'fw2',2:'fw3',3:'fw4',4:'fw5'}).reset_index(drop=True)
        s1=pd.merge(fw_t,pfw,how='outer').fillna(0)
        s2=pd.merge(s1,npfw,how='outer').fillna(0)
        s2['problem'] =s2['problem'].astype('int')
        s2['non-problem'] =s2['non-problem'].astype('int')
        s2['problem-ratio'] = round(s2['problem']/s2['counts'] *100,2)
        return s2

    else:
        fw_t= fw_df[[0,1,2,3,4]].value_counts().to_frame().rename(columns={0:'counts'}).reset_index().sort_values(0)\
        .rename(columns={0:'fw1',1:'fw2',2:'fw3',3:'fw4',4:'fw5'}).reset_index(drop=True)
        return fw_t

In [14]:
fw_ts = preprocessing_fw_update(tse,'test')
fw_tr = preprocessing_fw_update(tre,'train')

In [15]:
fw_ts=fw_ts.rename(columns={'counts':'test_c'})
fw_tr=fw_tr.rename(columns={'counts':'train_c'})
fw_totals = pd.merge(fw_tr,fw_ts,how='outer')[['fw1','fw2','fw3','fw4','fw5','problem','non-problem','problem-ratio','train_c','test_c']].fillna(0)

In [16]:
fw_totals

,fw1,fw2,fw3,fw4,fw5,problem,non-problem,problem-ratio,train_c,test_c
0,03.11.1141,0,0,0,0,0.0,1.0,0.00,1.0,2.0
1,03.11.1141,03.11.1167,0,0,0,1.0,0.0,100.00,1.0,0.0
2,03.11.1149,0,0,0,0,0.0,5.0,0.00,5.0,4.0
3,03.11.1149,03.11.1167,0,0,0,0.0,1.0,0.00,1.0,1.0
4,03.11.1167,04.33.1149,04.33.1261,0,0,2.0,0.0,100.00,2.0,0.0
5,03.11.1167,0,0,0,0,133.0,540.0,19.76,673.0,680.0
6,03.11.1167,04.33.1261,0,0,0,2.0,0.0,100.00,2.0,3.0
7,04.16.2641,04.33.1261,0,0,0,1.0,0.0,100.00,1.0,0.0
8,04.16.3345,04.33.1185,04.33.1261,0,0,1.0,0.0,100.00,1.0,0.0
9,04.16.3345,0,0,0,0,0.0,1.0,0.00,1.0,0.0


In [17]:
fw_totals.loc[fw_totals.train_c >100].sort_values('train_c',ascending=False)

,fw1,fw2,fw3,fw4,fw5,problem,non-problem,problem-ratio,train_c,test_c
42,04.22.1750,04.22.1778,0,0,0,1542.0,2564.0,37.55,4106.0,4184.0
59,04.33.1185,04.33.1261,0,0,0,940.0,2239.0,29.57,3179.0,3126.0
86,05.15.2138,0,0,0,0,662.0,2345.0,22.02,3007.0,3014.0
16,04.16.3553,0,0,0,0,569.0,1284.0,30.71,1853.0,1873.0
5,03.11.1167,0,0,0,0,133.0,540.0,19.76,673.0,680.0
63,04.33.1261,0,0,0,0,141.0,371.0,27.54,512.0,486.0
17,04.16.3553,04.16.3571,04.16.3553,0,0,173.0,204.0,45.89,377.0,420.0
21,04.16.3553,04.33.1261,0,0,0,250.0,17.0,93.63,267.0,242.0
44,04.22.1750,0,0,0,0,39.0,118.0,24.84,157.0,126.0
25,04.16.3553,04.33.1185,04.33.1261,0,0,135.0,10.0,93.10,145.0,153.0


In [66]:
### model - fwver -labling

mode = 'test'
target_df = tre
if mode =='train':
    first_num = 10000
    count_num =15000

else:
    first_num = 30000
    count_num =14999   


dp = target_df[['user_id','model_nm','fwver']]
unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

dp2 = target_df[['user_id','model_nm']]
unique_data2 =target_df[(dp2 !=dp2.shift(1)).sum(axis=1)>0]

fwver_total_dic ={}
for v in range(len(fwver_total)):
    fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
    


def fwver_tran(x):
    return fwver_total_dic[x]

fwver = np.zeros((count_num,24))
for idx in tqdm(unique_data.user_id.unique()):
    df_md =unique_data2.loc[unique_data2.user_id==idx].model_nm.values
    df_fw = unique_data.loc[unique_data.user_id==idx].fwver.values
    
    for md in range(len(df_md)):
        fwver[idx-first_num,md] = int(df_md[md][-1])+1
    
    for l in range(3,len(df_fw)+3):
        fwver[idx-first_num,l] =fwver_total_dic[df_fw[l-3]]

fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
fwver_total_dic_rev[0] =0
fwver_total_dic_rev2[0] = '04.22.1750'  #max 값


def fwver_tras_reverse(x):
    return fwver_total_dic_rev[x]

def fwver_tras_reverse2(x):
    return fwver_total_dic_rev2[x]

fw_df[3] =fw_df[3].apply(fwver_tras_reverse2)
fw_df[4] =fw_df[4].apply(fwver_tras_reverse)
fw_df[5] =fw_df[5].apply(fwver_tras_reverse)
fw_df[6] =fw_df[6].apply(fwver_tras_reverse)
fw_df[7] =fw_df[7].apply(fwver_tras_reverse)


fw_df = fw_df.rename(columns={0:'md1',1:'md2',2:'md3',3:'fw1',4:'fw2',5:'fw3',6:'fw4',7:'fw5'})
fw_df['user_id'] =fw_df['user_id']+10000

IndexError: ignored

In [19]:
def dr(num):
    user_id_idx = uid1.values[num]
    err_idx = df.loc[df.user_id ==user_id_idx]
    p_idx =df_p.loc[df_p.user_id ==user_id_idx]
    flow_idx = fw_df.loc[fw_df.user_id ==user_id_idx].iloc[:,:9]

    
    
    print("fwver_flow")
    display(flow_idx)
    print("problem time")
    display(p_idx)
    print('user err distribution')
    display(err_idx.errcode.value_counts().to_frame().T)

        
        
    fig ,axes = plt.subplots(2,1,figsize=(20,10))

    sns.scatterplot(data=err_idx,x='time' ,y='errtype',hue='fwver',s=7,ax=axes[0])
    if len(p_idx) !=0:
        for v in range(len(p_idx)):
            axes[0].axvline(p_idx.time.values[v],c='r',linestyle='--',label='prblem occur')
        
    start_date = pd.to_datetime("2020-11-01 11:00:00")
    for v in range(30):
        axes[0].axvline(start_date,c='y',linestyle='--',label='11:00')
        start_date +=datetime.timedelta(days=1)
        
        
    tqdf = tqq.loc[tqq.user_id ==user_id_idx]
    for tqs in range(len(tqdf)):
        axes[0].axvline(tqdf.time.values[tqs],c='b',linestyle='--',label='quality',alpha=0.5)
    
    axes[0].set_xlim(pd.to_datetime('2020-11-01'),pd.to_datetime('2020-11-30 23:00:00'))
    axes[0].set_title('errtype -fw, user_id :'+str(user_id_idx))
    handles, labels = axes[0].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axes[0].legend(by_label.values(), by_label.keys(),bbox_to_anchor=(1.05, 1), loc='upper left')
#     for idxs in err_idx.errcode.value_counts().to_frame().index:
#         code_df=err_idx.loc[err_idx.errcode ==idxs]


    sns.scatterplot(data=err_idx,x='time' ,y='errcode',hue='errtype',s=15,ax=axes[1], palette='tab20c')
    axes[1].set_xlim(pd.to_datetime('2020-11-01'),pd.to_datetime('2020-11-30 23:00:00'))
    axes[1].set_title('errcode -fw')
    axes[1].legend(title='errtype',bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

In [20]:
fw_tr.sort_values(['train_c','problem'],ascending=False)

,fw1,fw2,fw3,fw4,fw5,train_c,problem,non-problem,problem-ratio
42,04.22.1750,04.22.1778,0,0,0,4106,1542,2564,37.55
59,04.33.1185,04.33.1261,0,0,0,3179,940,2239,29.57
86,05.15.2138,0,0,0,0,3007,662,2345,22.02
16,04.16.3553,0,0,0,0,1853,569,1284,30.71
5,03.11.1167,0,0,0,0,673,133,540,19.76
63,04.33.1261,0,0,0,0,512,141,371,27.54
17,04.16.3553,04.16.3571,04.16.3553,0,0,377,173,204,45.89
21,04.16.3553,04.33.1261,0,0,0,267,250,17,93.63
44,04.22.1750,0,0,0,0,157,39,118,24.84
25,04.16.3553,04.33.1185,04.33.1261,0,0,145,135,10,93.10


In [21]:
def check_unique(col,df1,df2):
    def change_len(x):
        if len(x) ==10:
            return x[:5]
        else:
            return x
    print("about",col)
    if col !='fwvers':
        train_c = set(df1[col].unique())
        test_c  = set(df2[col].unique())
        print("train only in %s "%col,train_c-test_c)
        print("test only in %s "%col,test_c -train_c)
        total = (train_c | test_c)
    else:
        train_c = set(df1[col].apply(change_len).unique())
        test_c  = set(df2[col].apply(change_len).unique())
        print("train only in %s "%col,train_c-test_c)
        print("test only in %s "%col,test_c -train_c)
        total = (train_c | test_c)        
        
    print()
        
    return total

model_total=check_unique('model_nm',tre,tse)
errtype_total=check_unique('errtype',tre,tse)
fwver_total = check_unique('fwver',tre,tse)

about model_nm
train only in model_nm  set()
test only in model_nm  set()

about errtype
train only in errtype  set()
test only in errtype  set()

about fwver
train only in fwver  {'04.33.1095', '04.16.3345', '04.22.1442', '04.16.2641', '05.15.2122', '05.15.2090'}
test only in fwver  {'04.22.1170', '04.22.1478', '10.22.1780', '04.22.1772', '04.22.1448', '10.22.1770', '04.73.2577', '04.22.1608', '04.73.2569'}



In [69]:
num_df_user = 14999
num_errtype =42
first_index = 30000
num_model_change =2
types = 'test'
df = tse.copy()

In [70]:
#----------------------------------------------errtype
errtypes = np.zeros((num_df_user,num_errtype))
err = df[['user_id','errtype']].values

for inx, value in tqdm(err):
    errtypes[inx-first_index,value-1] +=1

In [24]:
pd.DataFrame(errtypes)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


In [71]:
# ---------------------------------------------model_change
v=df[['user_id','model_nm']]
getdf =~(v == v.shift(1))
logical =(getdf.user_id.apply(int) + getdf.model_nm.apply(int)) > 0
df_model_counts =v[logical]

In [72]:
v=df[['user_id','model_nm']]
getdf =~(v == v.shift(1))
logical =(getdf.user_id.apply(int) + getdf.model_nm.apply(int)) > 0
df_model_counts =v[logical]


def get_model_change_id(num):
    df_mc = df_model_counts.user_id.value_counts()
    df_mc_user=list(df_mc.loc[df_mc ==num].to_frame().index)
    sort_mc_user = df.loc[df.user_id.isin(df_mc_user)].drop_duplicates(['user_id','model_nm'],keep='last').drop_duplicates('user_id',keep='first').sort_values("time").user_id
    return sort_mc_user.values

one_m = get_model_change_id(1)
two_m = get_model_change_id(2)
thr_m = get_model_change_id(3)


model_n = np.zeros((num_df_user,num_model_change))
df['model_f'] =df['model_nm'].str[-1].astype('int')
one_df = df.loc[df.user_id.isin(one_m)][['user_id','model_f']].drop_duplicates().values
two_df =df.loc[df.user_id.isin(two_m)][['user_id','model_f']].drop_duplicates().reset_index(drop=True)
two_df['tf'] = two_df.index%2
two_df= two_df.pivot(index='user_id',columns='tf').reset_index().values
thr_df = df.loc[df.user_id.isin(thr_m)][['user_id','model_f']].drop_duplicates().values

for inx, value in tqdm(one_df):
    model_n[inx-first_index,0]  +=value
for inx, value1,value2 in tqdm(two_df):
    model_n[inx-first_index,0]  +=value1
    model_n[inx-first_index,1]  +=value2

for inx, value in tqdm(thr_df):
    model_n[inx-first_index,0]  +=value

In [27]:
pd.DataFrame(model_n)

,0,1
0,3.0,0.0
1,2.0,0.0
2,3.0,0.0
3,2.0,0.0
4,0.0,0.0
...,...,...
14995,2.0,0.0
14996,3.0,0.0
14997,0.0,0.0
14998,0.0,0.0


In [73]:
# ---------------------------------------------fw change
fwver_total_dic ={}
for v in range(len(fwver_total)):
    fwver_total_dic[sorted(list(fwver_total))[v]] = v+1

def fwver_tran(x):
    return fwver_total_dic[x]


df['ver_num'] = df['fwver'].apply(fwver_tran)
fwver_np = np.zeros((num_df_user,5))

v3=df[['user_id','ver_num']]
getdf =~(v3 == v3.shift(1))
logical =(getdf.user_id.apply(int) + getdf.ver_num.apply(int)) > 0
fwver_num=v3[logical]

fwver_num = fwver_num.reset_index(drop=True)
count =np.zeros(len(fwver_num),dtype=int)

for v in range(1,len(fwver_num)):
    if fwver_num.user_id.values[v-1] ==fwver_num.user_id.values[v]:
        count[v] = count[v-1] +1


fwver_num['count'] =count
fw_v = fwver_num.loc[fwver_num['count'].isin([0,1,2,3,4])].pivot(index='user_id',columns='count').reset_index().fillna(0).values
fw_v =fw_v.astype('int64')




for inx, v1,v2,v3,v4,v5 in tqdm(fw_v):
    fwver_np[inx-first_index,0] =v1
    fwver_np[inx-first_index,1] =v2
    fwver_np[inx-first_index,2] =v3
    fwver_np[inx-first_index,3] =v4
    fwver_np[inx-first_index,4] =v5

In [29]:
pd.DataFrame(fwver_np)

,0,1,2,3,4
0,39.0,0.0,0.0,0.0,0.0
1,25.0,26.0,0.0,0.0,0.0
2,39.0,0.0,0.0,0.0,0.0
3,25.0,26.0,0.0,0.0,0.0
4,18.0,20.0,0.0,0.0,0.0
...,...,...,...,...,...
14995,26.0,0.0,0.0,0.0,0.0
14996,39.0,0.0,0.0,0.0,0.0
14997,18.0,20.0,0.0,0.0,0.0
14998,18.0,20.0,0.0,0.0,0.0


In [74]:
#---------------------------------------------------------- errtype_ mean_std
df['week'] =df.time.dt.isocalendar().week

df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
datas = df[['user_id','errtype','week']]
df_=datas[['user_id','week','errtype']].value_counts().to_frame().reset_index()
df_ =df_.sort_values(['user_id','week']).rename(columns = {0:'counts'}).reset_index(drop=True)

df1 =df_.loc[df_.week ==44][['user_id','errtype','counts']].values
df2 =df_.loc[df_.week ==45][['user_id','errtype','counts']].values
df3 =df_.loc[df_.week ==46][['user_id','errtype','counts']].values
df4 =df_.loc[df_.week ==47][['user_id','errtype','counts']].values
df5 =df_.loc[df_.week ==48][['user_id','errtype','counts']].values

day_data = np.zeros((num_df_user,42,5))
for i, dfa in enumerate([df1,df2,df3,df4,df5]):
    for inx , val1 ,val2 in tqdm(dfa):
        day_data[:,:,i][inx-first_index,val1-1] = val2

m=day_data.mean(axis=2)
std=day_data.std(axis=2)       

In [31]:
pd.DataFrame(m)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,1.6,20.0,0.0,0.2,0.2,0.0,0.0,1.4,3.0,3.2,0.2,2.0,11.4,11.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.2,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,10.6,0.2,0.2,0.0,0.0,0.0,2.0,2.2,0.2,2.4,29.4,24.8,0.0,0.8,0.2,0.6,0.2,150.8,149.8,1.0,0.2,4.0,0.0,0.0,0.0,0.0,48.4,0.0,2.0,3.4,0.0,0.2,0.2,0.0,0.0,22.6,10.8,0.2
2,0.0,0.0,0.4,25.6,0.2,0.4,0.2,0.0,0.0,0.2,2.6,2.8,0.2,0.8,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,4.8,0.4,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.4,0.2,0.2,0.0,0.0,0.0,1.8,1.8,0.0,0.0,10.2,5.8,0.2,0.0,0.0,0.0,0.0,5.6,3.8,0.0,0.0,11.4,0.0,0.0,0.0,0.0,12.6,0.0,1.6,0.0,0.0,0.2,0.2,0.4,0.0,3.4,0.2,0.0
4,0.0,0.0,0.0,0.2,0.0,0.6,0.8,0.0,0.0,0.0,3.2,3.8,0.6,1.0,27.8,17.4,0.0,0.0,0.0,0.0,0.0,27.4,23.6,0.0,0.0,6.2,0.0,0.0,0.0,0.0,33.6,0.0,3.2,0.0,0.0,0.2,0.2,0.0,0.0,0.8,0.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.4,1.0,1.0,0.0,0.0,0.0,0.8,1.8,1.6,1.6,4.2,3.2,0.0,0.0,0.0,0.0,0.0,3.0,2.2,0.0,0.0,3.6,0.0,0.0,0.0,0.0,5.6,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.4,0.6
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,0.2,1.6,0.2,0.2,0.0,0.0,0.0,3.2,3.6,0.8,0.2,35.8,27.2,0.0,0.0,0.0,0.0,0.0,9.6,5.8,0.0,0.0,9.4,0.0,0.0,0.0,0.0,43.8,0.0,3.2,3.4,0.0,0.2,0.2,0.0,0.0,11.4,1.6,1.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.2,10.2,2.2,0.0,0.0,0.0,0.0,0.0,1.4,0.2,0.0,0.0,0.8,0.0,0.0,0.0,0.0,2.6,0.0,3.0,2.4,0.0,0.2,0.2,0.0,0.0,0.8,0.0,0.0


In [32]:
pd.DataFrame(std)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,1.624808,8.197561,0.000000,0.400000,0.400000,0.0,0.0,1.74356,2.097618,2.039608,0.400000,1.673320,5.083306,5.192302,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,2.712932,0.400000,0.400000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.000000,9.891410,0.400000,0.400000,0.0,0.0,0.00000,0.894427,0.748331,0.400000,3.006659,13.749182,12.765579,0.0,0.979796,0.4,0.8,0.4,286.689658,286.703610,2.0,0.4,3.162278,0.000000,0.000000,0.0,0.0,25.048752,0.0,0.894427,2.57682,0.0,0.4,0.4,0.000000,0.0,17.153425,7.984986,0.400000
2,0.0,0.0,0.800000,12.158947,0.400000,0.800000,0.400000,0.0,0.0,0.40000,1.019804,1.166190,0.400000,1.600000,5.761944,5.440588,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.8,0.0,2.039608,0.489898,0.489898,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.489898,0.400000,0.400000,0.0,0.0,0.00000,1.166190,1.166190,0.000000,0.000000,5.075431,2.481935,0.4,0.000000,0.0,0.0,0.0,3.200000,2.481935,0.0,0.0,7.144228,0.000000,0.000000,0.0,0.0,5.425864,0.0,1.019804,0.00000,0.0,0.4,0.4,0.489898,0.0,2.244994,0.400000,0.000000
4,0.0,0.0,0.000000,0.400000,0.000000,0.489898,0.748331,0.0,0.0,0.00000,1.166190,1.166190,0.489898,0.894427,11.788130,9.046546,0.0,0.000000,0.0,0.0,0.0,17.048167,17.095029,0.0,0.0,3.709447,0.000000,0.000000,0.0,0.0,19.314243,0.0,1.166190,0.00000,0.0,0.4,0.4,0.000000,0.0,0.979796,0.000000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.000000,0.000000,0.800000,1.549193,1.549193,0.0,0.0,0.00000,1.166190,2.712932,2.332381,2.332381,5.600000,4.118252,0.0,0.000000,0.0,0.0,0.0,4.647580,3.487119,0.0,0.0,4.800000,0.000000,0.000000,0.0,0.0,7.116179,0.0,1.166190,0.00000,0.0,0.0,0.0,0.000000,0.0,1.959592,2.800000,1.200000
14996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
14997,0.0,0.0,0.000000,0.400000,3.200000,0.400000,0.400000,0.0,0.0,0.00000,1.833030,2.059126,1.166190,0.400000,18.562327,14.469278,0.0,0.000000,0.0,0.0,0.0,5.351635,3.124100,0.0,0.0,5.238320,0.000000,0.000000,0.0,0.0,21.292252,0.0,1.833030,6.80000,0.0,0.4,0.4,0.000000,0.0,12.419340,3.200000,1.549193
14998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,1.095445,1.095445,0.000000,0.400000,3.762978,3.124100,0.0,0.000000,0.0,0.0,0.0,1.356466,0.400000,0.0,0.0,1.600000,0.000000,0.000000,0.0,0.0,3.072458,0.0,1.095445,4.80000,0.0,0.4,0.4,0.000000,0.0,1.600000,0.000000,0.000000


In [75]:
#----------------------------------------------------------- qulitydata
    
#----------------------------------------------------------- errtype_ mean_std _of hour

df['hour'] =df.time.dt.hour


df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
datas = df[['user_id','errtype','hour']]
df_=datas[['user_id','hour','errtype']].value_counts().to_frame().reset_index()
df_ =df_.sort_values(['user_id','hour']).rename(columns = {0:'counts'}).reset_index(drop=True)


day_data = np.zeros((num_df_user,42,24))
for i in range(24):
    dfa = df_.loc[df_['hour']==i][['user_id','errtype','counts']].values
    for inx , val1 ,val2 in tqdm(dfa):
        day_data[:,:,i][inx-first_index,val1-1] = val2

m2=day_data.mean(axis=2)
std2=day_data.std(axis=2)       

m2_1=day_data.max(axis=2)
#----------------------------------------------------------- errtype_ mean_std _of day

df['day'] =df.time.dt.day


df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
datas = df[['user_id','errtype','day']]
df_=datas[['user_id','day','errtype']].value_counts().to_frame().reset_index()
df_ =df_.sort_values(['user_id','day']).rename(columns = {0:'counts'}).reset_index(drop=True)


day_data = np.zeros((num_df_user,42,30))
for i in range(30):
    dfa = df_.loc[df_['day']==(i+1)][['user_id','errtype','counts']].values
    for inx , val1 ,val2 in tqdm(dfa):
        day_data[:,:,i][inx-first_index,val1-1] = val2

m3=day_data.mean(axis=2)
std3=day_data.std(axis=2)       

m3_1=day_data.max(axis=2) 

In [34]:
pd.DataFrame(m3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,0.266667,3.466667,0.000000,0.033333,0.033333,0.0,0.0,0.233333,0.500000,0.533333,0.033333,0.333333,1.966667,2.033333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.066667,0.033333,0.033333,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.000000,1.766667,0.033333,0.033333,0.0,0.0,0.000000,0.333333,0.366667,0.033333,0.500000,5.033333,4.266667,0.000000,0.133333,0.033333,0.1,0.033333,25.200000,25.033333,0.166667,0.033333,0.733333,0.000000,0.000000,0.0,0.000000,8.333333,0.0,0.333333,0.600000,0.0,0.033333,0.033333,0.000000,0.0,3.766667,1.866667,0.033333
2,0.0,0.0,0.066667,4.400000,0.033333,0.066667,0.033333,0.0,0.0,0.033333,0.433333,0.466667,0.033333,0.133333,1.733333,1.733333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.066667,0.000000,0.833333,0.066667,0.066667,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.066667,0.033333,0.033333,0.0,0.0,0.000000,0.300000,0.300000,0.000000,0.000000,1.733333,1.000000,0.033333,0.000000,0.000000,0.0,0.000000,0.933333,0.633333,0.000000,0.000000,1.966667,0.000000,0.000000,0.0,0.000000,2.166667,0.0,0.266667,0.000000,0.0,0.033333,0.033333,0.066667,0.0,0.566667,0.033333,0.000000
4,0.0,0.0,0.000000,0.033333,0.000000,0.100000,0.133333,0.0,0.0,0.000000,0.533333,0.633333,0.100000,0.166667,4.766667,3.033333,0.000000,0.000000,0.000000,0.0,0.000000,4.666667,3.966667,0.000000,0.000000,1.100000,0.000000,0.000000,0.0,0.000000,5.866667,0.0,0.533333,0.000000,0.0,0.033333,0.033333,0.000000,0.0,0.133333,0.000000,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.000000,0.000000,0.066667,0.166667,0.166667,0.0,0.0,0.000000,0.166667,0.333333,0.266667,0.266667,0.866667,0.600000,0.000000,0.000000,0.000000,0.0,0.000000,0.566667,0.400000,0.000000,0.000000,0.733333,0.000000,0.000000,0.0,0.000000,1.033333,0.0,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.300000,0.233333,0.133333
14996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,0.033333,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
14997,0.0,0.0,0.000000,0.033333,0.266667,0.033333,0.033333,0.0,0.0,0.000000,0.533333,0.600000,0.133333,0.033333,6.033333,4.600000,0.000000,0.000000,0.000000,0.0,0.000000,1.666667,0.966667,0.000000,0.000000,1.633333,0.000000,0.000000,0.0,0.000000,7.433333,0.0,0.533333,0.566667,0.0,0.033333,0.033333,0.000000,0.0,1.933333,0.266667,0.166667
14998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.500000,0.500000,0.000000,0.033333,1.700000,0.400000,0.000000,0.000000,0.000000,0.0,0.000000,0.233333,0.033333,0.000000,0.000000,0.133333,0.000000,0.000000,0.0,0.000000,0.466667,0.0,0.500000,0.400000,0.0,0.033333,0.033333,0.000000,0.0,0.200000,0.000000,0.000000


In [76]:
#------------------------------------------------------------- fwver flow
target_df = df
first_num = first_index
count_num =num_df_user

dp = target_df[['user_id','model_nm','fwver']]
unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

dp2 = target_df[['user_id','model_nm']]
unique_data2 =target_df[(dp2 !=dp2.shift(1)).sum(axis=1)>0]

fwver_total_dic ={}
for v in range(len(fwver_total)):
    fwver_total_dic[sorted(list(fwver_total))[v]] = v+1



def fwver_tran(x):
    return fwver_total_dic[x]

fwver = np.zeros((count_num,24))
for idx in tqdm(unique_data.user_id.unique()):
    df_md =unique_data2.loc[unique_data2.user_id==idx].model_nm.values
    df_fw = unique_data.loc[unique_data.user_id==idx].fwver.values

    for md in range(len(df_md)):
        fwver[idx-first_num,md] = int(df_md[md][-1])+1

    for l in range(3,len(df_fw)+3):
        fwver[idx-first_num,l] =fwver_total_dic[df_fw[l-3]]

fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
fwver_total_dic_rev[0] =0
fwver_total_dic_rev2[0] = '04.22.1750'  #max 값


def fwver_tras_reverse(x):
    return fwver_total_dic_rev[x]

def fwver_tras_reverse2(x):
    return fwver_total_dic_rev2[x]

fw_df[3] =fw_df[3].apply(fwver_tras_reverse2)
fw_df[4] =fw_df[4].apply(fwver_tras_reverse)
fw_df[5] =fw_df[5].apply(fwver_tras_reverse)
fw_df[6] =fw_df[6].apply(fwver_tras_reverse)
fw_df[7] =fw_df[7].apply(fwver_tras_reverse)


fw_df = fw_df.rename(columns={0:'md1',1:'md2',2:'md3',3:'fw1',4:'fw2',5:'fw3',6:'fw4',7:'fw5'})
fw_df['user_id'] =fw_df['user_id']+10000

pre_df=fw_df.iloc[:,:9]

md_flow = {str(x.astype("int")):(i+1) for i,x in enumerate(pre_df[['md1','md2','md3']].drop_duplicates().reset_index(drop=True).values)}
fw_flow = {str(x):(i+1) for i,x in enumerate(pre_df[['fw1','fw2','fw3','fw4','fw5']].drop_duplicates().reset_index(drop=True).values)}

def fw_change_counter(x):
    fwlst = []
    for v in ['fw1','fw2','fw3','fw4','fw5']:
        if x[v] ==0:
            pass
        else:
            fwlst +=[x[v]]

    if len(fwlst) ==len(list(set(fwlst))):
        return 0
    else:
        return 1

def check_problem(x):
    if x in p_user:
        return 1
    else:
        return 0

def md_flow_change(x):
    return md_flow[str(x[['md1','md2','md3']].values.astype("int"))]

def fw_flow_change(x):
    return fw_flow[str(x[['fw1','fw2','fw3','fw4','fw5']].values)]

def mean_str_fw_dum(x):
    fwlst = []
    for v in ['fw1','fw2','fw3','fw4','fw5']:
        if x[v] ==0:
            pass
        else:
            fwlst +=[int(x[v].replace('.',""))]
    return np.array(fwlst).mean()



def std_str_fw_dum(x):
    fwlst = []
    for v in ['fw1','fw2','fw3','fw4','fw5']:
        if x[v] ==0:
            pass
        else:
            fwlst +=[int(x[v].replace('.',""))]
    return np.array(fwlst).std()

pre_df=fw_df.iloc[:,:9]
pre_df['md_counts'] = pre_df[['md1','md2','md3']].astype('bool').sum(axis=1)
pre_df['fw_counts'] = pre_df[['fw1','fw2','fw3','fw4','fw5']].astype('bool').sum(axis=1)

pre_df['fw_change'] = pre_df.apply(fw_change_counter,axis=1)
pre_df['fw_flows'] = pre_df.apply(fw_flow_change,axis=1)
pre_df['md_flows'] = pre_df.apply(md_flow_change,axis=1)
pre_df['fw_mean'] = pre_df.apply(mean_str_fw_dum,axis=1)
pre_df['fw_std'] = pre_df.apply(std_str_fw_dum,axis=1)


fw_model_flow =pre_df.iloc[:,9:].values

In [36]:
pd.DataFrame(fw_model_flow)

,0,1,2,3,4,5,6
0,1.0,1.0,0.0,1.0,1.0,5152138.0,0.0
1,1.0,2.0,0.0,2.0,2.0,4331223.0,38.0
2,1.0,1.0,0.0,1.0,1.0,5152138.0,0.0
3,1.0,2.0,0.0,2.0,2.0,4331223.0,38.0
4,1.0,2.0,0.0,3.0,3.0,4221764.0,14.0
...,...,...,...,...,...,...,...
14995,1.0,1.0,0.0,15.0,2.0,4331261.0,0.0
14996,1.0,1.0,0.0,1.0,1.0,5152138.0,0.0
14997,1.0,2.0,0.0,3.0,3.0,4221764.0,14.0
14998,1.0,2.0,0.0,3.0,3.0,4221764.0,14.0


In [77]:
#------------------------------------------------------------------time term
first_num = first_index
count_num =num_df_user

time_term = np.zeros((count_num,4))
tre_t =df[['user_id','time']].drop_duplicates()

for v in tqdm(tre_t.user_id.unique()):
    test =tre_t.loc[tre_t.user_id ==v].time
    if len(test) <=2:
        time_term[v-first_num,0] = 0
        time_term[v-first_num,1] = 0
        time_term[v-first_num,2] = test.values[-1]-test.values[0]
        time_term[v-first_num,3] = len(test)
    else:
        time_term[v-first_num,0] = (test -test.shift(1)).max().total_seconds()
        time_term[v-first_num,1] = (test -test.shift(1)).min().total_seconds()
        time_term[v-first_num,2] = test.values[-1]-test.values[0]  
        time_term[v-first_num,3] = len(test)

dft = pd.DataFrame(time_term).copy()

dft[0] =dft[0]/3600
dft[2] =dft[2]/3600/24/10e8
dft[2] =np.where(dft[2].values==0,1,dft[2].values)
dft[5] = dft[0]/dft[3]*3600
dft[6] = dft[0]/24/dft[2]
time_term = dft.fillna(0).values

In [39]:
pd.DataFrame(time_term)

,0,1,2,3,4,5
0,22.359444,1.0,29.766863,285.0,282.435088,0.031298
1,21.823611,1.0,29.848970,2201.0,35.695139,0.030464
2,39.839444,1.0,29.758762,284.0,505.007042,0.055781
3,24.353889,1.0,29.027188,244.0,359.319672,0.034958
4,16.941111,1.0,29.847025,693.0,88.005772,0.023650
...,...,...,...,...,...,...
14995,28.871389,1.0,9.836030,152.0,683.796053,0.122303
14996,0.193333,276.0,0.011250,3.0,232.000000,0.716049
14997,18.521944,1.0,29.665394,741.0,89.985155,0.026015
14998,51.052222,1.0,29.664144,113.0,1626.442478,0.071709


In [78]:
#------------------------------------------------------------------- connetion err
err_df =df[['user_id','time','errcode']].dropna(axis=0)

df_con =err_df.loc[err_df.errcode.str.contains("connection")]
df_con['check_time'] = df_con.time.dt.date
df_con['check_hour'] =df_con.time.dt.hour

def connetion_df(con_df):
    day_con_err =con_df[['user_id','check_time']].value_counts().groupby("user_id").max()
    hour_con_err=con_df.groupby(['user_id','check_time','check_hour']).size().groupby("user_id").max()
    tenmin_con_err =con_df.set_index('time').groupby(['user_id','errcode']).resample("10min").size().groupby("user_id").max()
    con_trans =pd.concat([day_con_err,hour_con_err,tenmin_con_err],axis=1)
    return con_trans

total_conn_err=[connetion_df(df_con)]
for errs in tqdm(['connection timeout', 'connection fail to establish','connectionterminated by local host','connection fail for LMP response timout','L2CAP connection cancelled']):
    con_esta = df_con.loc[df_con.errcode.str.contains(errs)]
    total_conn_err.append(connetion_df(con_esta))

base_df =pd.DataFrame(range(first_num,first_num+count_num)).rename(columns={0:'user_id'}).set_index('user_id')

connetion_err_pre = pd.concat(total_conn_err+[base_df],axis=1).fillna(0).values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
pd.DataFrame(connetion_err_pre)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43.0,33.0,10.0,42.0,32.0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
#---------------------------------------------------------------------  main_fwver

tsed = df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
dfw = tsed[['user_id','fwver']]
fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

main_fw_ar = np.zeros((num_df_user,6))
for i,tgid in enumerate(tqdm(range(first_index,first_index+num_df_user))):

    tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
    tgidtotal = tsed.loc[tsed.user_id ==tgid]
    try:
        data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
        t1 =data.time
        if len(t1) %2 !=0:
            print('lenth error')
        time_delta = (t1-t1.shift(1)).dt.total_seconds()

        main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]
        main_fw_ar[i,0] = fwver_total_dic[main_fwver]
        main_fw_ar[i,1] =(time_delta[1::2].values).max().astype('float')/(time_delta.values[1:]).sum().astype('float')  #target fw workingtime / total
        if len(time_delta) ==1:
            main_fw_ar[i,2] =0  #min of change fwver time==0
            main_fw_ar[i,3] =0  #std of change fwver time ==0
            main_fw_ar[i,4] =0  #std
            main_fw_ar[i,5] =0  #variance
        else:
            main_fw_ar[i,2] =time_delta[::2].min()/3600 # min hours
            main_fw_ar[i,3] =time_delta[::2].std()/3600
            main_fw_ar[i,4] =time_delta[1::2].values.astype('float').std()/3600  #std running time of fw
            main_fw_ar[i,5] =(time_delta[1::2].values.astype('float')/3600).var()  #std running time of fw
    except:
        main_fw_ar[i,0] =0
        main_fw_ar[i,1] =0
        main_fw_ar[i,2] =0
        main_fw_ar[i,3] =0
        main_fw_ar[i,4] =0
        main_fw_ar[i,5] =0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


In [43]:
pd.DataFrame(main_fw_ar)

,0,1,2,3,4,5
0,39.0,1.000000,NaN,NaN,0.000000,0.000000
1,26.0,0.630035,0.093889,NaN,93.200694,8686.369445
2,39.0,1.000000,NaN,NaN,0.000000,0.000000
3,26.0,0.705696,0.095000,NaN,143.345972,20548.067752
4,18.0,0.805383,0.106944,NaN,218.808333,47877.086736
...,...,...,...,...,...,...
14995,26.0,1.000000,NaN,NaN,0.000000,0.000000
14996,39.0,1.000000,NaN,NaN,0.000000,0.000000
14997,18.0,0.601063,0.096667,NaN,72.002222,5184.320005
14998,20.0,0.932449,0.097778,NaN,307.926667,94818.832044


In [80]:
tred = df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
tred_all = df.dropna(axis=0).reset_index(drop=True)

main_fw_err_counts = np.zeros((num_df_user,84))
for i,tgid in enumerate(tqdm(range(first_index,num_df_user+first_index))):

    tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
    tgidtotal = tred_all.loc[tred_all.user_id ==tgid]
    
    try:
        data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
        t1 =data.time
        if len(t1) %2 !=0:
            print('lenth error')
        time_delta = (t1-t1.shift(1)).dt.total_seconds()

        main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]


        main_date = tgidtotal.loc[tgidtotal.fwver ==main_fwver][['time','errtype']]
        main_date['date'] =main_date.time.dt.date
        main_time_del  =(main_date.time.dt.date.values[-1] -main_date.time.dt.date.values[0]).days
        date_first= main_date.time.dt.date.values[0]
        err_count_main = np.zeros((42,main_time_del+1))
        for n in  range(main_time_del+1):
            lcdf = main_date.loc[main_date.date ==date_first]
            for errtype in lcdf.errtype.values:
                err_count_main[errtype-1,n] +=1
            date_first += datetime.timedelta(days=1)

        main_fw_err_counts[i,:42] =     err_count_main.mean(axis=1)
        main_fw_err_counts[i,42:] =     err_count_main.std(axis=1)

    except:
        main_fw_err_counts[i,:] = 0 

In [45]:
pd.DataFrame(main_fw_err_counts)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83
0,0.0,0.0,0.266667,3.466667,0.000000,0.033333,0.033333,0.0,0.0,0.233333,0.500000,0.533333,0.033333,0.333333,1.966667,2.033333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249444,0.179505,0.179505,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.000000,0.000000,2.315789,0.000000,0.000000,0.0,0.0,0.000000,0.368421,0.368421,0.000000,0.368421,4.526316,3.736842,0.0,0.210526,0.052632,0.157895,0.052632,39.526316,39.526316,0.263158,0.052632,...,0.613784,0.223297,0.488085,0.223297,159.703465,159.469307,0.713929,0.223297,1.093927,0.000000,0.000000,0.0,0.0,3.537688,0.0,0.482376,1.093927,0.0,0.223297,0.223297,0.000000,0.0,2.394592,1.427859,0.223297
2,0.0,0.0,0.066667,4.400000,0.033333,0.066667,0.033333,0.0,0.0,0.033333,0.433333,0.466667,0.033333,0.133333,1.733333,1.733333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359011,0.000000,0.372678,0.249444,0.249444,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.0,0.0,0.000000,0.380952,0.380952,0.000000,0.000000,1.857143,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.952381,0.571429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.361938,1.136889,0.000000,0.000000,0.903508,0.000000,0.000000,0.0,0.0,1.401328,0.0,0.471405,0.000000,0.0,0.212959,0.212959,0.212959,0.0,0.722179,0.212959,0.000000
4,0.0,0.0,0.000000,0.040000,0.000000,0.120000,0.160000,0.0,0.0,0.000000,0.520000,0.640000,0.120000,0.200000,4.640000,2.960000,0.0,0.000000,0.000000,0.000000,0.000000,4.800000,4.280000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,8.380931,8.632589,0.000000,0.000000,0.463033,0.000000,0.000000,0.0,0.0,4.280187,0.0,0.574108,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.542586,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.000000,0.000000,0.181818,0.454545,0.454545,0.0,0.0,0.000000,0.454545,0.909091,0.727273,0.727273,2.363636,1.636364,0.0,0.000000,0.000000,0.000000,0.000000,1.545455,1.090909,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.233151,0.995859,0.000000,0.000000,0.852803,0.000000,0.000000,0.0,0.0,1.748671,0.0,0.497930,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.113404,1.493789,0.481046
14996,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
14997,0.0,0.0,0.000000,0.052632,0.000000,0.052632,0.052632,0.0,0.0,0.000000,0.473684,0.526316,0.052632,0.052632,5.526316,4.315789,0.0,0.000000,0.000000,0.000000,0.000000,1.157895,0.736842,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.039390,0.713929,0.000000,0.000000,0.759063,0.000000,0.000000,0.0,0.0,3.817876,0.0,0.751729,3.796047,0.0,0.000000,0.000000,0.000000,0.0,3.162278,0.000000,0.000000
14998,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.500000,0.500000,0.000000,0.035714,1.678571,0.428571,0.0,0.000000,0.000000,0.000000,0.000000,0.214286,0.035714,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.489690,0.185577,0.000000,0.000000,0.742307,0.000000,0.000000,0.0,0.0,0.823754,0.0,0.566947,2.226922,0.0,0.185577,0.185577,0.000000,0.

In [81]:
a = pd.DataFrame(model_n)
a.rename(columns = lambda x: str(x) + '_model_b', inplace = True)
b = pd.DataFrame(fwver_np)
b.rename(columns = lambda x: str(x) + '_fwver_b', inplace = True)

c = pd.DataFrame(m)
c.rename(columns = lambda x: str(x) + '_err_mean_byday_b', inplace = True)
d = pd.DataFrame(std)
c.rename(columns = lambda x: str(x) + '_err_std_byday_b', inplace = True)

e = pd.DataFrame(m2)
e.rename(columns = lambda x: str(x) + '_err_mean_byhour_b', inplace = True)
f = pd.DataFrame(std2)
f.rename(columns = lambda x: str(x) + '_err_std_byhour_b', inplace = True)
g = pd.DataFrame(m2_1)
g.rename(columns = lambda x: str(x) + '_err_max_byhour_b', inplace = True)


h = pd.DataFrame(m3)
h.rename(columns = lambda x: str(x) + '_err_mean_inday_b', inplace = True)
i = pd.DataFrame(std3)
i.rename(columns = lambda x: str(x) + '_err_std_inday_b', inplace = True)
j = pd.DataFrame(m3_1)
j.rename(columns = lambda x: str(x) + '_err_max_inday_b', inplace = True)

k = pd.DataFrame(fw_model_flow)
k.rename(columns = lambda x: str(x) + '_fw_flow_b', inplace = True)
l = pd.DataFrame(time_term)
l.rename(columns = lambda x: str(x) + '_fw_flow_time_b', inplace = True)

m = pd.DataFrame(connetion_err_pre)
m.rename(columns = lambda x: str(x) + '_con_err_b', inplace = True)
n = pd.DataFrame(main_fw_ar)
n.rename(columns = lambda x: str(x) + '_main_fw_err_b', inplace = True)
o = pd.DataFrame(main_fw_err_counts)
o.rename(columns = lambda x: str(x) + '_main_fw_ercount_b', inplace = True)

In [82]:
a.to_csv('a.csv', index = False)
!cp a.csv "drive/My Drive/"
b.to_csv('b.csv', index = False)
!cp b.csv "drive/My Drive/"
c.to_csv('c.csv', index = False)
!cp c.csv "drive/My Drive/"
d.to_csv('d.csv', index = False)
!cp d.csv "drive/My Drive/"
e.to_csv('e.csv', index = False)
!cp e.csv "drive/My Drive/"
f.to_csv('f.csv', index = False)
!cp f.csv "drive/My Drive/"
g.to_csv('g.csv', index = False)
!cp g.csv "drive/My Drive/"
h.to_csv('h.csv', index = False)
!cp h.csv "drive/My Drive/"
i.to_csv('i.csv', index = False)
!cp i.csv "drive/My Drive/"
j.to_csv('j.csv', index = False)
!cp j.csv "drive/My Drive/"
k.to_csv('k.csv', index = False)
!cp k.csv "drive/My Drive/"
l.to_csv('l.csv', index = False)
!cp l.csv "drive/My Drive/"
m.to_csv('m.csv', index = False)
!cp m.csv "drive/My Drive/"
n.to_csv('n.csv', index = False)
!cp n.csv "drive/My Drive/"
o.to_csv('o.csv', index = False)
!cp o.csv "drive/My Drive/"

In [ ]:
sets.to_csv("트레인_데이콘_보너스.csv", index = False)
!cp 트레인_데이콘_보너스.csv "drive/My Drive/"

In [ ]:
sets = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o], axis = 1)
sets

,0_model_b,1_model_b,0_fwver_b,1_fwver_b,2_fwver_b,3_fwver_b,4_fwver_b,0_err_mean_byday_b_err_std_byday_b,1_err_mean_byday_b_err_std_byday_b,2_err_mean_byday_b_err_std_byday_b,3_err_mean_byday_b_err_std_byday_b,4_err_mean_byday_b_err_std_byday_b,5_err_mean_byday_b_err_std_byday_b,6_err_mean_byday_b_err_std_byday_b,7_err_mean_byday_b_err_std_byday_b,8_err_mean_byday_b_err_std_byday_b,9_err_mean_byday_b_err_std_byday_b,10_err_mean_byday_b_err_std_byday_b,11_err_mean_byday_b_err_std_byday_b,12_err_mean_byday_b_err_std_byday_b,13_err_mean_byday_b_err_std_byday_b,14_err_mean_byday_b_err_std_byday_b,15_err_mean_byday_b_err_std_byday_b,16_err_mean_byday_b_err_std_byday_b,17_err_mean_byday_b_err_std_byday_b,...,59_main_fw_ercount_b,60_main_fw_ercount_b,61_main_fw_ercount_b,62_main_fw_ercount_b,63_main_fw_ercount_b,64_main_fw_ercount_b,65_main_fw_ercount_b,66_main_fw_ercount_b,67_main_fw_ercount_b,68_main_fw_ercount_b,69_main_fw_ercount_b,70_main_fw_ercount_b,71_main_fw_ercount_b,72_main_fw_ercount_b,73_main_fw_ercount_b,74_main_fw_ercount_b,75_main_fw_ercount_b,76_main_fw_ercount_b,77_main_fw_ercount_b,78_main_fw_ercount_b,79_main_fw_ercount_b,80_main_fw_ercount_b,81_main_fw_ercount_b,82_main_fw_ercount_b,83_main_fw_ercount_b
0,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,20.0,0.0,0.2,0.2,0.0,0.0,1.4,3.0,3.2,0.2,2.0,11.4,11.8,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249444,0.179505,0.179505,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,2.0,0.0,25.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.6,0.2,0.2,0.0,0.0,0.0,2.0,2.2,0.2,2.4,29.4,24.8,0.0,0.8,...,0.613784,0.223297,0.488085,0.223297,159.703465,159.469307,0.713929,0.223297,1.093927,0.000000,0.000000,0.0,0.0,3.537688,0.0,0.482376,1.093927,0.0,0.223297,0.223297,0.000000,0.0,2.394592,1.427859,0.223297
2,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,25.6,0.2,0.4,0.2,0.0,0.0,0.2,2.6,2.8,0.2,0.8,10.0,10.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359011,0.000000,0.372678,0.249444,0.249444,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,2.0,0.0,25.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,0.2,0.0,0.0,0.0,1.8,1.8,0.0,0.0,10.2,5.8,0.2,0.0,...,0.000000,0.000000,0.000000,0.000000,1.361938,1.136889,0.000000,0.000000,0.903508,0.000000,0.000000,0.0,0.0,1.401328,0.0,0.471405,0.000000,0.0,0.212959,0.212959,0.212959,0.0,0.722179,0.212959,0.000000
4,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.6,0.8,0.0,0.0,0.0,3.2,3.8,0.6,1.0,27.8,17.4,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,8.380931,8.632589,0.000000,0.000000,0.463033,0.000000,0.000000,0.0,0.0,4.280187,0.0,0.574108,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.542586,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.0,1.0,0.0,0.0,0.0,0.8,1.8,1.6,1.6,4.2,3.2,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.233151,0.995859,0.000000,0.000000,0.852803,0.000000,0.000000,0.0,0.0,1.748671,0.0,0.497930,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.113404,1.493789,0.481046
14996,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
14997,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,1.6,0.2,0.2,0.0,0.0,0.0,3.2,3.6,0.8,0.2,35.8,27.2,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.039390,0.713929,0.000000,0.000000,0.759063,0.000000,0.000000,0.0,0.0,3.817876,0.0,0.751729,3.796047,0.0,0.000000,0.000000,0.000000,0.0,3.162278,0.000000,0.000000
14998,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
def dataset_trans(df,types,Num_df_user,Num_errtype,First_index):
    
    
    num_df_user = Num_df_user
    num_errtype =Num_errtype
    first_index = First_index
    num_model_change =2

    
    #----------------------------------------------errtype
    errtypes = np.zeros((num_df_user,num_errtype))
    err = df[['user_id','errtype']].values

    for inx, value in tqdm(err):
        errtypes[inx-first_index,value-1] +=1


    # ---------------------------------------------model_change
    v=df[['user_id','model_nm']]
    getdf =~(v == v.shift(1))
    logical =(getdf.user_id.apply(int) + getdf.model_nm.apply(int)) > 0
    df_model_counts =v[logical]


    def get_model_change_id(num):
        df_mc = df_model_counts.user_id.value_counts()
        df_mc_user=list(df_mc.loc[df_mc ==num].to_frame().index)
        sort_mc_user = df.loc[df.user_id.isin(df_mc_user)].drop_duplicates(['user_id','model_nm'],keep='last').drop_duplicates('user_id',keep='first').sort_values("time").user_id
        return sort_mc_user.values

    one_m = get_model_change_id(1)
    two_m = get_model_change_id(2)
    thr_m = get_model_change_id(3)


    model_n = np.zeros((num_df_user,num_model_change))
    df['model_f'] =df['model_nm'].str[-1].astype('int')
    one_df = df.loc[df.user_id.isin(one_m)][['user_id','model_f']].drop_duplicates().values
    two_df =df.loc[df.user_id.isin(two_m)][['user_id','model_f']].drop_duplicates().reset_index(drop=True)
    two_df['tf'] = two_df.index%2
    two_df= two_df.pivot(index='user_id',columns='tf').reset_index().values
    thr_df = df.loc[df.user_id.isin(thr_m)][['user_id','model_f']].drop_duplicates().values

    for inx, value in tqdm(one_df):
        model_n[inx-first_index,0]  +=value
    for inx, value1,value2 in tqdm(two_df):
        model_n[inx-first_index,0]  +=value1
        model_n[inx-first_index,1]  +=value2

    for inx, value in tqdm(thr_df):
        model_n[inx-first_index,0]  +=value


    # ---------------------------------------------fw change
    fwver_total_dic ={}
    for v in range(len(fwver_total)):
        fwver_total_dic[sorted(list(fwver_total))[v]] = v+1

    def fwver_tran(x):
        return fwver_total_dic[x]


    df['ver_num'] = df['fwver'].apply(fwver_tran)
    fwver_np = np.zeros((num_df_user,5))

    v3=df[['user_id','ver_num']]
    getdf =~(v3 == v3.shift(1))
    logical =(getdf.user_id.apply(int) + getdf.ver_num.apply(int)) > 0
    fwver_num=v3[logical]

    fwver_num = fwver_num.reset_index(drop=True)
    count =np.zeros(len(fwver_num),dtype=int)

    for v in range(1,len(fwver_num)):
        if fwver_num.user_id.values[v-1] ==fwver_num.user_id.values[v]:
            count[v] = count[v-1] +1


    fwver_num['count'] =count
    fw_v = fwver_num.loc[fwver_num['count'].isin([0,1,2,3,4])].pivot(index='user_id',columns='count').reset_index().fillna(0).values
    fw_v =fw_v.astype('int64')

    
    
    
    for inx, v1,v2,v3,v4,v5 in tqdm(fw_v):
        fwver_np[inx-first_index,0] =v1
        fwver_np[inx-first_index,1] =v2
        fwver_np[inx-first_index,2] =v3
        fwver_np[inx-first_index,3] =v4
        fwver_np[inx-first_index,4] =v5

    #---------------------------------------------------------- errtype_ mean_std
    df['week'] =df.time.dt.isocalendar().week

    df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','week']]
    df_=datas[['user_id','week','errtype']].value_counts().to_frame().reset_index()
    df_ =df_.sort_values(['user_id','week']).rename(columns = {0:'counts'}).reset_index(drop=True)

    df1 =df_.loc[df_.week ==44][['user_id','errtype','counts']].values
    df2 =df_.loc[df_.week ==45][['user_id','errtype','counts']].values
    df3 =df_.loc[df_.week ==46][['user_id','errtype','counts']].values
    df4 =df_.loc[df_.week ==47][['user_id','errtype','counts']].values
    df5 =df_.loc[df_.week ==48][['user_id','errtype','counts']].values

    day_data = np.zeros((num_df_user,42,5))
    for i, dfa in enumerate([df1,df2,df3,df4,df5]):
        for inx , val1 ,val2 in tqdm(dfa):
            day_data[:,:,i][inx-first_index,val1-1] = val2

    m=day_data.mean(axis=2)
    std=day_data.std(axis=2)       
        

        
        
    #----------------------------------------------------------- qulitydata
        
    #----------------------------------------------------------- errtype_ mean_std _of hour

    df['hour'] =df.time.dt.hour


    df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','hour']]
    df_=datas[['user_id','hour','errtype']].value_counts().to_frame().reset_index()
    df_ =df_.sort_values(['user_id','hour']).rename(columns = {0:'counts'}).reset_index(drop=True)


    day_data = np.zeros((num_df_user,42,24))
    for i in range(24):
        dfa = df_.loc[df_['hour']==i][['user_id','errtype','counts']].values
        for inx , val1 ,val2 in tqdm(dfa):
            day_data[:,:,i][inx-first_index,val1-1] = val2

    m2=day_data.mean(axis=2)
    std2=day_data.std(axis=2)       

    m2_1=day_data.max(axis=2)
    #----------------------------------------------------------- errtype_ mean_std _of day

    df['day'] =df.time.dt.day


    df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<=pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','day']]
    df_=datas[['user_id','day','errtype']].value_counts().to_frame().reset_index()
    df_ =df_.sort_values(['user_id','day']).rename(columns = {0:'counts'}).reset_index(drop=True)


    day_data = np.zeros((num_df_user,42,30))
    for i in range(30):
        dfa = df_.loc[df_['day']==(i+1)][['user_id','errtype','counts']].values
        for inx , val1 ,val2 in tqdm(dfa):
            day_data[:,:,i][inx-first_index,val1-1] = val2

    m3=day_data.mean(axis=2)
    std3=day_data.std(axis=2)       

    m3_1=day_data.max(axis=2) 
    
    
    
    
    
    #------------------------------------------------------------- fwver flow
    target_df = df
    first_num = first_index
    count_num =num_df_user

    dp = target_df[['user_id','model_nm','fwver']]
    unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

    dp2 = target_df[['user_id','model_nm']]
    unique_data2 =target_df[(dp2 !=dp2.shift(1)).sum(axis=1)>0]

    fwver_total_dic ={}
    for v in range(len(fwver_total)):
        fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
    


    def fwver_tran(x):
        return fwver_total_dic[x]

    fwver = np.zeros((count_num,24))
    for idx in tqdm(unique_data.user_id.unique()):
        df_md =unique_data2.loc[unique_data2.user_id==idx].model_nm.values
        df_fw = unique_data.loc[unique_data.user_id==idx].fwver.values

        for md in range(len(df_md)):
            fwver[idx-first_num,md] = int(df_md[md][-1])+1

        for l in range(3,len(df_fw)+3):
            fwver[idx-first_num,l] =fwver_total_dic[df_fw[l-3]]

    fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

    fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
    fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
    fwver_total_dic_rev[0] =0
    fwver_total_dic_rev2[0] = '04.22.1750'  #max 값


    def fwver_tras_reverse(x):
        return fwver_total_dic_rev[x]

    def fwver_tras_reverse2(x):
        return fwver_total_dic_rev2[x]

    fw_df[3] =fw_df[3].apply(fwver_tras_reverse2)
    fw_df[4] =fw_df[4].apply(fwver_tras_reverse)
    fw_df[5] =fw_df[5].apply(fwver_tras_reverse)
    fw_df[6] =fw_df[6].apply(fwver_tras_reverse)
    fw_df[7] =fw_df[7].apply(fwver_tras_reverse)


    fw_df = fw_df.rename(columns={0:'md1',1:'md2',2:'md3',3:'fw1',4:'fw2',5:'fw3',6:'fw4',7:'fw5'})
    fw_df['user_id'] =fw_df['user_id']+10000

    pre_df=fw_df.iloc[:,:9]

    md_flow = {str(x.astype("int")):(i+1) for i,x in enumerate(pre_df[['md1','md2','md3']].drop_duplicates().reset_index(drop=True).values)}
    fw_flow = {str(x):(i+1) for i,x in enumerate(pre_df[['fw1','fw2','fw3','fw4','fw5']].drop_duplicates().reset_index(drop=True).values)}

    def fw_change_counter(x):
        fwlst = []
        for v in ['fw1','fw2','fw3','fw4','fw5']:
            if x[v] ==0:
                pass
            else:
                fwlst +=[x[v]]

        if len(fwlst) ==len(list(set(fwlst))):
            return 0
        else:
            return 1

    def check_problem(x):
        if x in p_user:
            return 1
        else:
            return 0

    def md_flow_change(x):
        return md_flow[str(x[['md1','md2','md3']].values.astype("int"))]

    def fw_flow_change(x):
        return fw_flow[str(x[['fw1','fw2','fw3','fw4','fw5']].values)]

    def mean_str_fw_dum(x):
        fwlst = []
        for v in ['fw1','fw2','fw3','fw4','fw5']:
            if x[v] ==0:
                pass
            else:
                fwlst +=[int(x[v].replace('.',""))]
        return np.array(fwlst).mean()



    def std_str_fw_dum(x):
        fwlst = []
        for v in ['fw1','fw2','fw3','fw4','fw5']:
            if x[v] ==0:
                pass
            else:
                fwlst +=[int(x[v].replace('.',""))]
        return np.array(fwlst).std()

    pre_df=fw_df.iloc[:,:9]
    pre_df['md_counts'] = pre_df[['md1','md2','md3']].astype('bool').sum(axis=1)
    pre_df['fw_counts'] = pre_df[['fw1','fw2','fw3','fw4','fw5']].astype('bool').sum(axis=1)

    pre_df['fw_change'] = pre_df.apply(fw_change_counter,axis=1)
    pre_df['fw_flows'] = pre_df.apply(fw_flow_change,axis=1)
    pre_df['md_flows'] = pre_df.apply(md_flow_change,axis=1)
    pre_df['fw_mean'] = pre_df.apply(mean_str_fw_dum,axis=1)
    pre_df['fw_std'] = pre_df.apply(std_str_fw_dum,axis=1)


    fw_model_flow =pre_df.iloc[:,9:].values    


    #------------------------------------------------------------------time term
    first_num = first_index
    count_num =num_df_user
    
    time_term = np.zeros((count_num,4))
    tre_t =df[['user_id','time']].drop_duplicates()

    for v in tqdm(tre_t.user_id.unique()):
        test =tre_t.loc[tre_t.user_id ==v].time
        if len(test) <=2:
            time_term[v-first_num,0] = 0
            time_term[v-first_num,1] = 0
            time_term[v-first_num,2] = test.values[-1]-test.values[0]
            time_term[v-first_num,3] = len(test)
        else:
            time_term[v-first_num,0] = (test -test.shift(1)).max().total_seconds()
            time_term[v-first_num,1] = (test -test.shift(1)).min().total_seconds()
            time_term[v-first_num,2] = test.values[-1]-test.values[0]  
            time_term[v-first_num,3] = len(test)

    dft = pd.DataFrame(time_term).copy()

    dft[0] =dft[0]/3600
    dft[2] =dft[2]/3600/24/10e8
    dft[2] =np.where(dft[2].values==0,1,dft[2].values)
    dft[5] =dft[0]/dft[3]
    dft[5] = dft[0]/dft[3]*3600
    dft[6] = dft[0]/24/dft[2]
    time_term = dft.fillna(0).values

    #------------------------------------------------------------------- connetion err
    err_df =df[['user_id','time','errcode']].dropna(axis=0)

    df_con =err_df.loc[err_df.errcode.str.contains("connection")]
    df_con['check_time'] = df_con.time.dt.date
    df_con['check_hour'] =df_con.time.dt.hour

    def connetion_df(con_df):
        day_con_err =con_df[['user_id','check_time']].value_counts().groupby("user_id").max()
        hour_con_err=con_df.groupby(['user_id','check_time','check_hour']).size().groupby("user_id").max()
        tenmin_con_err =con_df.set_index('time').groupby(['user_id','errcode']).resample("10min").size().groupby("user_id").max()
        con_trans =pd.concat([day_con_err,hour_con_err,tenmin_con_err],axis=1)
        return con_trans

    total_conn_err=[connetion_df(df_con)]
    for errs in tqdm(['connection timeout', 'connection fail to establish','connectionterminated by local host','connection fail for LMP response timout','L2CAP connection cancelled']):
        con_esta = df_con.loc[df_con.errcode.str.contains(errs)]
        total_conn_err.append(connetion_df(con_esta))

    base_df =pd.DataFrame(range(first_num,first_num+count_num)).rename(columns={0:'user_id'}).set_index('user_id')

    connetion_err_pre = pd.concat(total_conn_err+[base_df],axis=1).fillna(0).values
    
    
    
    
    
    #---------------------------------------------------------------------  main_fwver

    
    
    tsed = df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
    dfw = tsed[['user_id','fwver']]
    fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

    main_fw_ar = np.zeros((num_df_user,6))
    for i,tgid in enumerate(tqdm(range(first_index,first_index+num_df_user))):

        tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
        tgidtotal = tsed.loc[tsed.user_id ==tgid]
        try:
            data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
            t1 =data.time
            if len(t1) %2 !=0:
                print('lenth error')
            time_delta = (t1-t1.shift(1)).dt.total_seconds()

            main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]
            main_fw_ar[i,0] = fwver_total_dic[main_fwver]
            main_fw_ar[i,1] =(time_delta[1::2].values).max().astype('float')/(time_delta.values[1:]).sum().astype('float')  #target fw workingtime / total
            if len(time_delta) ==1:
                main_fw_ar[i,2] =0  #min of change fwver time==0
                main_fw_ar[i,3] =0  #std of change fwver time ==0
                main_fw_ar[i,4] =0  #std
                main_fw_ar[i,5] =0  #variance
            else:
                main_fw_ar[i,2] =time_delta[::2].min()/3600 # min hours
                main_fw_ar[i,3] =time_delta[::2].std()/3600
                main_fw_ar[i,4] =time_delta[1::2].values.astype('float').std()/3600  #std running time of fw
                main_fw_ar[i,5] =(time_delta[1::2].values.astype('float')/3600).var()  #std running time of fw
        except:
            main_fw_ar[i,0] =0
            main_fw_ar[i,1] =0
            main_fw_ar[i,2] =0
            main_fw_ar[i,3] =0
            main_fw_ar[i,4] =0
            main_fw_ar[i,5] =0

            
            
            
    #---------------------------------------------------------------------         

 
    
    
    tred = df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
    tred_all = df.dropna(axis=0).reset_index(drop=True)

    main_fw_err_counts = np.zeros((num_df_user,84))
    for i,tgid in enumerate(tqdm(range(first_index,num_df_user+first_index))):

        tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
        tgidtotal = tred_all.loc[tred_all.user_id ==tgid]
        
        try:
            data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
            t1 =data.time
            if len(t1) %2 !=0:
                print('lenth error')
            time_delta = (t1-t1.shift(1)).dt.total_seconds()

            main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]


            main_date = tgidtotal.loc[tgidtotal.fwver ==main_fwver][['time','errtype']]
            main_date['date'] =main_date.time.dt.date
            main_time_del  =(main_date.time.dt.date.values[-1] -main_date.time.dt.date.values[0]).days
            date_first= main_date.time.dt.date.values[0]
            err_count_main = np.zeros((42,main_time_del+1))
            for n in  range(main_time_del+1):
                lcdf = main_date.loc[main_date.date ==date_first]
                for errtype in lcdf.errtype.values:
                    err_count_main[errtype-1,n] +=1
                date_first += datetime.timedelta(days=1)

            main_fw_err_counts[i,:42] =     err_count_main.mean(axis=1)
            main_fw_err_counts[i,42:] =     err_count_main.std(axis=1)

        except:
            main_fw_err_counts[i,:] = 0 


    #train data의 경우 label 까지 출력
    # if types == 'train':
#         def tf(x):
#             if x>=1:
#                 return 1
#             else:
#                 return 0
#         df['tf'] =df['count_e'].apply(tf)
#         label=df[['user_id','tf']].drop_duplicates().values[:,1].reshape(-1,1)
# #         return [fwver_np,model_n, errtypes, m, std ,m2,std2,m2_1,m3,m3_1,std3,fw_model_flow , time_term   ,label]
#         return [fwver_np,model_n, errtypes, m, std ,m2,std2,m2_1,m3,m3_1,std3,fw_model_flow , time_term ,connetion_err_pre  ,main_fw_ar,main_fw_err_counts,label]
# #         return [fwver_np,model_n, m, std ,m2,std2,m2_1,m3,m3_1,std3     ,label]  #testing

#     else:
# #         return [fwver_np,model_n, errtypes, m ,std ,m2,std2,m2_1,m3,m3_1,std3,fw_model_flow  ,time_term      ]
    return [fwver_np,model_n, errtypes, m ,std ,m2,std2,m2_1,m3,m3_1,std3,fw_model_flow  ,time_term  ,connetion_err_pre  ,main_fw_ar,main_fw_err_counts  ]
#         return [fwver_np,model_n, m, std ,m2,std2,m2_1,m3,m3_1,std3     ]#testing

In [ ]:
%%time
datalist =dataset_trans(tre,'train',15000,42,10000)
train_data = np.concatenate(tuple(datalist[:-1]),axis=1)
label = datalist[-1]

print(train_data.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_


(15000, 422)
CPU times: user 31min 18s, sys: 43.7 s, total: 32min 2s
Wall time: 21min 12s


In [ ]:
%%time
datatest =dataset_trans(tse,'test',14999,42,30000)
test_data = np.concatenate(tuple(datatest),axis=1)
print(test_data.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:365: RuntimeWarning: invalid value encountered in double_


(14999, 506)
CPU times: user 32min 2s, sys: 45.4 s, total: 32min 48s
Wall time: 21min 29s


In [ ]:
tr_idx = [x for x in range(10000, 25000)]
ts_idx = [x for x in range(30000, 44999)]

In [ ]:
dft = pd.DataFrame(test_data, index =ts_idx)
dfs = pd.DataFrame(train_data, index = tr_idx)

In [ ]:
dft.rename(columns = lambda x: str(x) + '_dacon', inplace = True)
dfs.rename(columns = lambda x: str(x) + '_dacon', inplace = True)

In [ ]:
dft

,0_dacon,1_dacon,2_dacon,3_dacon,4_dacon,5_dacon,6_dacon,7_dacon,8_dacon,9_dacon,10_dacon,11_dacon,12_dacon,13_dacon,14_dacon,15_dacon,16_dacon,17_dacon,18_dacon,19_dacon,20_dacon,21_dacon,22_dacon,23_dacon,24_dacon,...,481_dacon,482_dacon,483_dacon,484_dacon,485_dacon,486_dacon,487_dacon,488_dacon,489_dacon,490_dacon,491_dacon,492_dacon,493_dacon,494_dacon,495_dacon,496_dacon,497_dacon,498_dacon,499_dacon,500_dacon,501_dacon,502_dacon,503_dacon,504_dacon,505_dacon
30000,7.0,9.0,7.0,26.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,...,0.000000,0.0,0.000000,0.0,62.862207,90.655606,0.771580,0.000000,0.496904,0.000000,0.000000,0.0,0.0,2.222839,0.000000,0.496904,0.000000,0.000000,0.188853,0.188853,0.000000,0.377705,2.705479,0.000000,0.0
30001,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.476095,0.179505,0.179505,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
30002,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,1.573166,1.427229,0.384615,0.000000,0.615385,0.000000,0.000000,0.0,0.0,3.618861,0.000000,0.571772,0.000000,0.000000,0.000000,0.000000,0.000000,2.324299,3.540342,0.192308,0.0
30003,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,...,0.000000,0.0,0.000000,0.0,0.208299,0.208299,0.000000,0.000000,0.208299,0.208299,0.208299,0.0,0.0,3.267989,0.000000,0.287480,2.916185,0.416598,0.000000,0.000000,0.000000,0.000000,1.032529,0.456813,0.0
30004,7.0,23.0,26.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.199826,0.199826,0.0,0.0,5.728674,13.927391,0.623610,1.198958,0.199826,0.000000,0.000000,0.199826,0.000000,2.234515,1.831438,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,...,0.000000,0.0,0.000000,0.0,4.793630,5.676854,0.000000,0.000000,0.179505,0.000000,0.000000,0.0,0.0,3.177875,0.359011,0.561743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.627059,1.640122,0.0
44995,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.156424,0.000000,0.500000,0.185577,0.000000,0.185577,0.185577,0.000000,0.000000,0.699854,0.822980,0.0
44996,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,...,0.000000,0.0,0.000000,0.0,2.049255,4.055023,0.000000,0.000000,0.686232,0.000000,0.000000,0.0,0.0,3.333703,0.000000,0.482376,123.859115,0.000000,0.000000,0.000000,0.223297,0.000000,10.440704,0.223297,0.0
44997,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,...,0.188853,0.0,0.188853,0.0,237.101635,286.932727,0.000000,0.566558,0.698813,0.000000,0.000000,0.0,0.0,1.790081,0.000000,0.566558,0.000000,0.000000,0.000000,0.000000,0.415740,0.000000,1.666255,1.023724,0.0


In [ ]:
dfs

,0_dacon,1_dacon,2_dacon,3_dacon,4_dacon,5_dacon,6_dacon,7_dacon,8_dacon,9_dacon,10_dacon,11_dacon,12_dacon,13_dacon,14_dacon,15_dacon,16_dacon,17_dacon,18_dacon,19_dacon,20_dacon,21_dacon,22_dacon,23_dacon,24_dacon,...,397_dacon,398_dacon,399_dacon,400_dacon,401_dacon,402_dacon,403_dacon,404_dacon,405_dacon,406_dacon,407_dacon,408_dacon,409_dacon,410_dacon,411_dacon,412_dacon,413_dacon,414_dacon,415_dacon,416_dacon,417_dacon,418_dacon,419_dacon,420_dacon,421_dacon
10000,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,...,0.031298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
10001,25.0,26.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,...,0.030464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.630035,0.093889,NaN,93.200694,8686.369445
10002,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,...,0.055781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
10003,25.0,26.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,...,0.034958,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.705696,0.095000,NaN,143.345972,20548.067752
10004,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,...,0.023650,43.0,33.0,10.0,42.0,32.0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.805383,0.106944,NaN,218.808333,47877.086736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,26.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,...,0.122303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,1.000000,NaN,NaN,0.000000,0.000000
24996,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.716049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
24997,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,...,0.026015,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.601063,0.096667,NaN,72.002222,5184.320005
24998,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,...,0.071709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.932449,0.097778,NaN,307.926667,94818.832044


In [ ]:
sets

,0_model_b,1_model_b,0_fwver_b,1_fwver_b,2_fwver_b,3_fwver_b,4_fwver_b,0_err_mean_byday_b_err_std_byday_b,1_err_mean_byday_b_err_std_byday_b,2_err_mean_byday_b_err_std_byday_b,3_err_mean_byday_b_err_std_byday_b,4_err_mean_byday_b_err_std_byday_b,5_err_mean_byday_b_err_std_byday_b,6_err_mean_byday_b_err_std_byday_b,7_err_mean_byday_b_err_std_byday_b,8_err_mean_byday_b_err_std_byday_b,9_err_mean_byday_b_err_std_byday_b,10_err_mean_byday_b_err_std_byday_b,11_err_mean_byday_b_err_std_byday_b,12_err_mean_byday_b_err_std_byday_b,13_err_mean_byday_b_err_std_byday_b,14_err_mean_byday_b_err_std_byday_b,15_err_mean_byday_b_err_std_byday_b,16_err_mean_byday_b_err_std_byday_b,17_err_mean_byday_b_err_std_byday_b,...,59_main_fw_ercount_b,60_main_fw_ercount_b,61_main_fw_ercount_b,62_main_fw_ercount_b,63_main_fw_ercount_b,64_main_fw_ercount_b,65_main_fw_ercount_b,66_main_fw_ercount_b,67_main_fw_ercount_b,68_main_fw_ercount_b,69_main_fw_ercount_b,70_main_fw_ercount_b,71_main_fw_ercount_b,72_main_fw_ercount_b,73_main_fw_ercount_b,74_main_fw_ercount_b,75_main_fw_ercount_b,76_main_fw_ercount_b,77_main_fw_ercount_b,78_main_fw_ercount_b,79_main_fw_ercount_b,80_main_fw_ercount_b,81_main_fw_ercount_b,82_main_fw_ercount_b,83_main_fw_ercount_b
0,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,20.0,0.0,0.2,0.2,0.0,0.0,1.4,3.0,3.2,0.2,2.0,11.4,11.8,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.249444,0.179505,0.179505,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,2.0,0.0,25.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.6,0.2,0.2,0.0,0.0,0.0,2.0,2.2,0.2,2.4,29.4,24.8,0.0,0.8,...,0.613784,0.223297,0.488085,0.223297,159.703465,159.469307,0.713929,0.223297,1.093927,0.000000,0.000000,0.0,0.0,3.537688,0.0,0.482376,1.093927,0.0,0.223297,0.223297,0.000000,0.0,2.394592,1.427859,0.223297
2,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,25.6,0.2,0.4,0.2,0.0,0.0,0.2,2.6,2.8,0.2,0.8,10.0,10.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359011,0.000000,0.372678,0.249444,0.249444,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,2.0,0.0,25.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,0.2,0.0,0.0,0.0,1.8,1.8,0.0,0.0,10.2,5.8,0.2,0.0,...,0.000000,0.000000,0.000000,0.000000,1.361938,1.136889,0.000000,0.000000,0.903508,0.000000,0.000000,0.0,0.0,1.401328,0.0,0.471405,0.000000,0.0,0.212959,0.212959,0.212959,0.0,0.722179,0.212959,0.000000
4,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.6,0.8,0.0,0.0,0.0,3.2,3.8,0.6,1.0,27.8,17.4,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,8.380931,8.632589,0.000000,0.000000,0.463033,0.000000,0.000000,0.0,0.0,4.280187,0.0,0.574108,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.542586,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.0,1.0,0.0,0.0,0.0,0.8,1.8,1.6,1.6,4.2,3.2,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.233151,0.995859,0.000000,0.000000,0.852803,0.000000,0.000000,0.0,0.0,1.748671,0.0,0.497930,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.113404,1.493789,0.481046
14996,3.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
14997,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,1.6,0.2,0.2,0.0,0.0,0.0,3.2,3.6,0.8,0.2,35.8,27.2,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.039390,0.713929,0.000000,0.000000,0.759063,0.000000,0.000000,0.0,0.0,3.817876,0.0,0.751729,3.796047,0.0,0.000000,0.000000,0.000000,0.0,3.162278,0.000000,0.000000
14998,0.0,0.0,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
sets.to_csv("트레인_데이콘_보너스.csv", index = False)
!cp 트레인_데이콘_보너스.csv "drive/My Drive/"

In [ ]:
dfs.to_csv("Dacon_bonus_train.csv", index = False)
!cp Dacon_bonus_train.csv "drive/My Drive/"

dft.to_csv("Dacon_bonus_test.csv", index = False)
!cp Dacon_bonus_test.csv "drive/My Drive/"

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421
10000,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,...,0.031298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
10001,25.0,26.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,...,0.030464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.630035,0.093889,NaN,93.200694,8686.369445
10002,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,...,0.055781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
10003,25.0,26.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,...,0.034958,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.705696,0.095000,NaN,143.345972,20548.067752
10004,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,...,0.023650,43.0,33.0,10.0,42.0,32.0,10.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.805383,0.106944,NaN,218.808333,47877.086736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,26.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,...,0.122303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,1.000000,NaN,NaN,0.000000,0.000000
24996,39.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.716049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,1.000000,NaN,NaN,0.000000,0.000000
24997,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,...,0.026015,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.601063,0.096667,NaN,72.002222,5184.320005
24998,18.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,...,0.071709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.932449,0.097778,NaN,307.926667,94818.832044
